<a href="https://colab.research.google.com/github/anuraglahon16/AI-Engineering/blob/main/llama3_with_DSPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimizing llama 3 prompts with DSPY





In [ ]:
!pip install transformers accelerate bitsandbytes dspy-ai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 197.5/197.5 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 33.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.4/520.4 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 40.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 407.5/407.5 kB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 87.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━

In [ ]:
#hugging face log in
from huggingface_hub import notebook_login

notebook_login()

Note that to run the following code, you must have got access to Llama 2's weights and have an access token from Hugging Face. You can find instructions on the model cards on the hugging face hub: https://huggingface.co/meta-llama/Llama-2-7b-chat-hf


In [ ]:
# Import necessary libraries and load the LLaMA 3 model and tokenizer
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
quantization_config = BitsAndBytesConfig(load_in_4bit=True,bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16)


model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", quantization_config=quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
import dspy
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric

# # Load the GSM8K dataset and split it into train and dev sets
gms8k = GSM8K()
gsm8k_trainset, gsm8k_devset = gms8k.train[:30], gms8k.dev[:50]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

100%|██████████| 1319/1319 [00:00<00:00, 30576.61it/s]


In [18]:
# Retrieve specific example from the dev set
prompt = gms8k.dev[28]
model_inputs = tokenizer(prompt["question"], return_tensors="pt").to("cuda:0")
output = model.generate(**model_inputs,max_new_tokens=100)

print(tokenizer.decode(output[0], skip_special_tokens=True))
print("\nCorrect answer:",prompt["answer"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


A pack of dogs found a pile of bones in the dumpster behind the butcher shop.  One by one, the dogs crawled into the dumpster, grabbed a mouthful of bones, and ran off with their newfound treasure.  In all, there were 5 dogs.  The first dog carried off 3 bones.  The second dog carried off 1 less bone than the first dog.  The third dog carried off twice as many as the second dog.  The fourth dog carried off one bone, and the fifth dog carried off twice the number of bones as the fourth dog carried, leaving no remaining bones in the dumpster.  How many bones were in the original pile of bones?  A) 15 B) 16 C) 17 D) 18 E) 19
Answer:
The first dog carried off 3 bones.  The second dog carried off 2 bones.  The third dog carried off 4 bones.  The fourth dog carried off 1 bone.  The fifth dog carried off 2 bones.  In total, the dogs carried off 3 + 2 + 4 + 1 + 2 = 12 bones.

Correct answer: 12


In [ ]:
# Set up the LLaMA 3 model as the language model for DSPY
from dsp.modules.lm import LM
def openai_to_hf(**kwargs):
    hf_kwargs = {}
    for k, v in kwargs.items():
        if k == "n":
            hf_kwargs["num_return_sequences"] = v
        elif k == "frequency_penalty":
            hf_kwargs["repetition_penalty"] = 1.0 - v
        elif k == "presence_penalty":
            hf_kwargs["diversity_penalty"] = v
        elif k == "max_tokens":
            hf_kwargs["max_new_tokens"] = v
        elif k == "model":
            pass
        else:
            hf_kwargs[k] = v

    return hf_kwargs

class HFModel(LM):
    def __init__(self, model:AutoModelForCausalLM, tokenizer:AutoTokenizer, **kwargs):
        """wrapper for Hugging Face models

        Args:
            model (AutoModelForCausalLM): HF model identifier to load and use
            tokenizer: AutoTokenizer
        """
        super().__init__(model)
        self.model = model
        self.tokenizer = tokenizer
        self.drop_prompt_from_output = True
        self.history = []
        self.is_client = False
        self.device = model.device
        self.kwargs = {
            "temperature": 0.3,
            "max_new_tokens": 300,
        }

    def basic_request(self, prompt, **kwargs):
        raw_kwargs = kwargs
        kwargs = {**self.kwargs, **kwargs}
        response = self._generate(prompt, **kwargs)

        history = {
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
            "raw_kwargs": raw_kwargs,
        }
        self.history.append(history)

        return response

    def _generate(self, prompt, **kwargs):
        kwargs = {**openai_to_hf(**self.kwargs), **openai_to_hf(**kwargs)}
        if isinstance(prompt, dict):
            try:
                prompt = prompt['messages'][0]['content']
            except (KeyError, IndexError, TypeError):
                print("Failed to extract 'content' from the prompt.")
        inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)

        # print(kwargs)
        outputs = self.model.generate(**inputs, **kwargs)
        if self.drop_prompt_from_output:
            input_length = inputs.input_ids.shape[1]
            outputs = outputs[:, input_length:]
        completions = [
            {"text": c}
            for c in self.tokenizer.batch_decode(outputs, skip_special_tokens=True)
        ]
        response = {
            "prompt": prompt,
            "choices": completions,
        }
        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        assert only_completed, "for now"
        assert return_sorted is False, "for now"

        if kwargs.get("n", 1) > 1 or kwargs.get("temperature", 0.0) > 0.1:
            kwargs["do_sample"] = True


        response = self.request(prompt, **kwargs)
        return [c["text"] for c in response["choices"]]

In [ ]:
# Set up the LM
llama = HFModel(model,tokenizer)
dspy.settings.configure(lm=llama)

In [ ]:
# Define the QASignature and CoT (Chain-of-Thought)
class QASignature(dspy.Signature):
    ("""You are given a question and answer"""
    """and you must think step by step to answer the question. """
    """Only include the answer as the output.""")
    question = dspy.InputField(desc="A math question")
    answer = dspy.OutputField(desc="An answer that is a number")

class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought(QASignature)

    def forward(self, question):
        return self.prog(question=question)

In [ ]:
# Optimize the CoT module using BootstrapFewShot
from dspy.teleprompt import BootstrapFewShot

# Set up the optimizer
config = dict(max_bootstrapped_demos=2)

# Optimize
teleprompter = BootstrapFewShot(metric=gsm8k_metric, **config)
optimized_cot = teleprompter.compile(CoT(), trainset=gsm8k_trainset, valset=gsm8k_devset)

  0%|          | 0/30 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
 13%|█▎        | 4/30 [01:49<11:53, 27.42s/it]

Bootstrapped 2 full traces after 5 examples in round 0.


In [ ]:
from dspy.evaluate import Evaluate

# Set up the evaluator, which can be used multiple times.
evaluate = Evaluate(devset=gsm8k_devset, metric=gsm8k_metric, num_threads=4, display_progress=True, display_table=0)

# Evaluate our `optimized_cot` program.
evaluate(optimized_cot)

  0%|          | 0/50 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Average Metric: 39 / 50  (78.0): 100%|██████████| 50/50 [34:33<00:00, 41.47s/it]

Average Metric: 39 / 50  (78.0%)


78.0

In [ ]:
llama.inspect_history(n=1)





You are given a question and answerand you must think step by step to answer the question. Only include the answer as the output.

---

Follow the following format.

Question: A math question
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: An answer that is a number

---

Question: Bridget counted 14 shooting stars in the night sky. Reginald counted two fewer shooting stars than did Bridget, but Sam counted four more shooting stars than did Reginald. How many more shooting stars did Sam count in the night sky than was the average number of shooting stars observed for the three of them?
Reasoning: Let's think step by step in order to find the answer. We know that Bridget counted 14 shooting stars. Reginald counted 2 fewer than Bridget, so he counted 14 - 2 = 12. Sam counted 4 more than Reginald, so he counted 12 + 4 = 16. To find the average, we add the number of shooting stars counted by each person and divide by 3. So the average is (14 + 12 +

In [ ]:
# compare to our regular COT
evaluate(CoT())

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 0/50 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Average Metric: 0.0 / 2  (0.0):   4%|▍         | 2/50 [02:46<54:48, 68.50s/it]   Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error for example in dev set: 		 Required 'max_tokens' or 'max_output_tokens' not specified in settings.


Average Metric: 1.0 / 6  (16.7):  12%|█▏        | 6/50 [05:34<36:17, 49.48s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error for example in dev set: 		 Required 'max_tokens' or 'max_output_tokens' not specified in settings.


Average Metric: 16.0 / 26  (61.5):  52%|█████▏    | 26/50 [19:25<18:14, 45.61s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Error for example in dev set: 		 Required 'max_tokens' or 'max_output_tokens' not specified in settings.


Average Metric: 31.0 / 50  (62.0): 100%|██████████| 50/50 [34:21<00:00, 41.22s/it]

Average Metric: 31.0 / 50  (62.0%)


62.0

In [ ]:
class Zeroshot(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.Predict(QASignature)

    def forward(self, question):
        return self.prog(question=question)

In [ ]:
evaluate(Zeroshot())

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  0%|          | 0/50 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Average Metric: 10 / 50  (20.0): 100%|██████████| 50/50 [34:29<00:00, 41.39s/it]

Average Metric: 10 / 50  (20.0%)


20.0

**The optimized CoT model (78.0% accuracy) outperforms both the regular CoT model (62.0% accuracy) and the zero-shot learning model (20.0% accuracy) on the math problem-solving task using the LLaMA 3 model and the GSM8K dataset. Prompt optimization using DSPY significantly improves the model's performance compared to the regular CoT and zero-shot learning approaches.**